# Efektywne programowanie w języku Python 

## wykład 10

![alt text](images/django.png "Guido van Rossum")

## Zacznijmy od początku

Wersje

> As a final heads up, Django 1.11 is likely to be the last version to support Python 2.7 as it will be supported until the end of Python 2 upstream support in 2020.

![alt text](images/release-roadmap.e844db08610e.png "ob")

## MVC vs. MTV

### Django wygląda, jakby było frameworkiem MVC, ale Kontroler nazywacie „widokiem” a Widok „szablonem”. Dlaczego nie używacie standardowych nazw?

![alt text](images/introduction-to-django-6-638.jpg "ob")

![alt text](images/day-3-django-flow.png "Guido van Rossum")

## Modele

![alt text](images/day-2-orm.png "ob")


In [ ]:
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': 'mydatabase',
    }
}

In [ ]:
DATABASES = {
    'default': {
        'ENGINE':'django.db.backends.postgresql_psycopg2',
        'NAME': 'mydatabase',
        'USER': 'mydatabaseuser',
        'PASSWORD': 'mypassword',
        'HOST': '127.0.0.1',
        'PORT': '5432',
    }
}

In [ ]:
from django.db import models

class Person(models.Model):
    first_name = models.CharField(max_length=30)
    last_name = models.CharField(max_length=30)

In [ ]:
CREATE TABLE myapp_person (
    "id" serial NOT NULL PRIMARY KEY,
    "first_name" varchar(30) NOT NULL,
    "last_name" varchar(30) NOT NULL
);

In [ ]:
Person.objects.filter(
...     first_name='John',
...     last_name='Bee')

Możemy oczwiście komplikować sobie modele

In [ ]:
poll = models.ForeignKey(
    Poll,
    on_delete=models.CASCADE,
    verbose_name="the related poll",
)
sites = models.ManyToManyField(Site, verbose_name="list of sites")
place = models.OneToOneField(
    Place,
    on_delete=models.CASCADE,
    verbose_name="related place",
)

wszystkie typy pól odnajdziemy tutaj [https://docs.djangoproject.com/en/1.11/ref/models/fields/](https://docs.djangoproject.com/en/1.11/ref/models/fields/)

## Template

Dowolny tekst otoczony podwójną parą nawiasów klamrowych (np. `{{ person_name }}`) jest zmienną. Oznacza "wstaw w tym miejscu wartość zmiennej o podanej nazwie". 

Dowolny tekst otoczony nawiasem klamrowym i znakiem procentu (np. `{% if ordered_warranty %}`) jest znacznikiem szablonu - znacznik  informuje po prostu system szablonowy o potrzebie wykonania pewnego działania).

Dowolny tekst otoczony nawiasem klamrowym i znakiem hasz (np. `{# komentarz #}`) jest komentarzem - tekst pomiędzy tym tagiem jest ignorowany.


![alt text](images/Obraz1.png "Guido van Rossum")

## View

Oparte o funkcje

In [ ]:
from django.http import HttpResponse
import datetime

def about(request):
    now = datetime.datetime.now()
    html = "<html><body>O nas. Teraz jest %s.</body></html>" % now
    return HttpResponse(html)

Oparte o funkcje (bardziej zaawansowany przykład)

In [ ]:
from django.http import HttpResponse
from django.template import loader

from .models import Question


def index(request):
    latest_question_list = Question.objects.order_by('-pub_date')[:5]
    template = loader.get_template('polls/index.html')
    context = {
        'latest_question_list': latest_question_list,
    }
    return HttpResponse(template.render(context, request))

In [ ]:
from django.conf.urls import url
from dfrom some_app.views import import about

urlpatterns = [
    url(r'^about/$', about),
]

Oparte o klasy generyczne (bez modyfikacji)

In [ ]:
from django.conf.urls import url
from django.views.generic import TemplateView

urlpatterns = [
    url(r'^about/$', TemplateView.as_view(template_name="about.html")),
]

Oparte o klasy generyczne (z modyfikacją)

In [ ]:
# some_app/views.py
from django.views.generic import TemplateView

class AboutView(TemplateView):
    template_name = "about.html"

In [ ]:
# urls.py
from django.conf.urls import url
from some_app.views import AboutView

urlpatterns = [
    url(r'^about/$', AboutView.as_view()),
]

## URL

In [ ]:
# URLconf
from django.conf.urls import patterns, url

urlpatterns = patterns('',
    url(r'^blog/$', 'blog.views.page'),
    url(r'^blog/page(?P<num>\d+)/$', 'blog.views.page'),
)

# View (in blog/views.py)
def page(request, num="1"):
    # Output the appropriate page of blog entries, according to num.
    ...

http://blog.com/blog/1

## Formularze

Możemy to zrobic od zera

In [ ]:
from django import forms

class ContactForm(forms.Form):
    subject = forms.CharField(max_length=100)
    message = forms.CharField()
    sender = forms.EmailField()
    cc_myself = forms.BooleanField(required=False)

In [ ]:
from django.shortcuts import render
from django.http import HttpResponseRedirect

def contact(request):
    if request.method == 'POST': # If the form has been submitted...
        # ContactForm was defined in the the previous section
        form = ContactForm(request.POST) # A form bound to the POST data
        if form.is_valid(): # All validation rules pass
            # Process the data in form.cleaned_data
            # ...
            return HttpResponseRedirect('/thanks/') # Redirect after POST
    else:
        form = ContactForm() # An unbound form

    return render(request, 'contact.html', {
        'form': form,
    })


Możemy też skorzystać z gotowca jeśli mamy już 'jakiś' model

In [ ]:
from django.forms import ModelForm

class AuthorForm(ModelForm):
    class Meta:
        model = Author
        fields = '__all__'

Źródła:
- [slash4 trainings: Learn Django in 4 hours](http://slash4.net/learn-django/) - materiał obowiązkowy na zajęcia
- [Strona projektu](https://www.djangoproject.com)
- [Django Packages](https://djangopackages.org)
- https://github.com/kmisztal/django_simple_project - mój projekt na githubie
- http://slides.com/dominikwronski/wit-ep2-django-introduction/fullscreen
